In [ ]:
#Connect to Google drive
from google.colab import drive

drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
#This is not required unless you want to delete the runtime
from google.colab import runtime
runtime.unassign()

In [ ]:
import json
import re
with open("/content/drive/MyDrive/COMS574-Introduction_To_Machine_Learning/News_Category_Dataset_v3.json", "r",encoding='utf-8') as f:
    text = f.read()

#parse json in the files to extract text
pattern = '\{[^{}]*\}'

matches = re.findall(pattern, text)

In [ ]:
#size of dataset
len(matches)

209527

In [ ]:
!pip install swifter
!pip install inflect
import inflect
from multiprocessing import Pool, cpu_count
#Also import all nltk libraries
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag, ne_chunk
from nltk.tree import Tree
nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('maxent_ne_chunker_tab')
nltk.download('punkt_tab')

nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import words
#download and import dictionary of words
nltk.download('words')
word_list = words.words()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=30d775793c7a00936f47b0c058d2975e1151728acdb0be87b0ad05c4498e154a
  Stored in directory: /root/.cache/pip/wheels/ef/7f/bd/9bed48f078f3ee1fa75e0b29b6e0335ce1cb03a38d3443b3a3
Successfully built swifter


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package maxent_ne_chunker_tab to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker_tab.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nl

In [ ]:
#create pandas dataframe out of json file
import pandas as pd
json_objs=[]
for item in matches:
  try:
     json_obj = json.loads(item)
     json_objs.append(json_obj)
  except json.JSONDecodeError:
     print(f"Could not decode: {item}")

df = pd.DataFrame(json_objs)


Could not decode: {Today's Buddha Doodle}
Could not decode: {Today's Buddha Doodle}
Could not decode: {This was originally posted to The Screaming", "date": "2013-08-08"}
Could not decode: {Newlywed}
Could not decode: {sic}
Could not decode: {EM}


In [ ]:
#The number of rows in dataframe
print(len(df))

209521


In [ ]:
#category counts of ecah category
value_counts_column = df['category'].value_counts()
print(value_counts_column)

category
POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9899
STYLE & BEAUTY     9813
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3652
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1396
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1073
EDUCATI

In [ ]:
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [ ]:
#Just taking a subset of 15 categories which are relevant and have good amount of data
#Max 5000 rows are selected for eah category.
#This is to avoid bias towards particular category.
df_subset = pd.DataFrame()
categories_shortlisted =['POLITICS','WELLNESS','TRAVEL','STYLE & BEAUTY','SCIENCE','WOMEN','CRIME','BUSINESS','RELIGION'\
,'TECH','U.S. NEWS','WORLD NEWS','ENTERTAINMENT','HEALTHY LIVING','FOOD & DRINK']

for category in categories_shortlisted:
  temp_df = df[df['category'] == category]
  if len(temp_df) >=5000:
    temp_df = temp_df.sample(frac=5000/len(temp_df));
  df_subset = pd.concat([df_subset,temp_df],ignore_index=True)

print(len(df_subset))

58697


In [ ]:
import copy
setwords = set(word_list)
p = inflect.engine()
#If its a dictionary word then just return the lowercase so its not identified as named entity.
#This is required for named entity recognition as the dictionary words should not be identified as named entities
def checkifItsDictionaryWord(token):
   word = copy.deepcopy(token)
   #Inorder to check if its a dictionary word, do some preprocessing
   #removal of 's or s' for any special characters.
   #also remove the inflections such as plural form.
   if "'s" in word:
     word.replace("'s","")
   elif "s'" in word:
     word.replace("s'","")

   res = p.singular_noun(word)
   #print(res)
   if res!=False:
      word = res

   word = re.sub(r'[^a-zA-Z0-9]', '', word)
   if word.lower() in setwords:
     del word;
     return token.lower()
   else:
     del word;
     return token



In [ ]:
checkifItsDictionaryWord('Cars')

'cars'

In [ ]:
import inflect
import numpy as np

In [ ]:
#Convert words to lower case if its a dictionary word so that its not falsely identified as named entity.
def convertWordsToLowerCase(sentence):
   tokens = [checkifItsDictionaryWord(token)  for token in sentence.split()]

   sentence = " ".join([token.lower() if token.lower() in setwords else token for token in tokens])
   return sentence

sentence = "Cars are best vehicles for Kate's travel"
print(convertWordsToLowerCase(sentence))

cars are best vehicles for Kate's travel


In [ ]:
#Since all the words in headline are starting with CAPS we want to change it to lower case if its a dictionary word
#So that its not falsely identified as named entity
df_subset['headline'] = df_subset['headline'].apply(convertWordsToLowerCase)

In [ ]:
df_subset['headline'][4]

"Obama hits Birthers in white house Correspondents' dinner speech"

In [ ]:
#Also apply the above to short description too
df_subset['short_description'] = df_subset['short_description'].apply(convertWordsToLowerCase)

In [ ]:
#Finally concatenate the headline and short description.
df_subset['news_text'] = df_subset['headline']+ " "+df_subset['short_description']

In [ ]:
df_subset['headline'][0]

"teacher removed from classroom over white nationalist Podcast says it's satire"

In [ ]:
#Here is the code for named entity recogtion. This has to be plugged with preprocession.
def getNamedEntities(text):
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)
    chunked_tree = ne_chunk(pos_tags)

    bio_tags = []
    for subtree in chunked_tree:
        if isinstance(subtree, Tree):  # Named entity found
            entity_name = [token for token, pos in subtree.leaves()]
            entity_type = subtree.label()

            for i, token in enumerate(entity_name):
                bio_tag = f"B-{entity_type}" if i == 0 else f"I-{entity_type}"
                bio_tags.append((token, bio_tag))
        else:
            token, pos = subtree
            bio_tags.append((token, "O"))
    return bio_tags

def verifyIfEntitiesArePresent(text):

    named_entities = getNamedEntities(text)
    #print(named_entities)
    # Example usage
    #text = "Putin was the 44th President of the United States on 23-05-2001."
    #text = "Steve Jobs and Steve Wozniak founded Apple on April 1, 1976 in Cupertino, California"
    #named_entities = extract_named_entities_with_bio(text)
    #print(named_entities)
    personPresent = 0
    organizationPresent = 0
    gpePresent = 0
    locationPresent = 0
    for i in range(0,len(named_entities)):
      if named_entities[i][1] == 'B-PERSON':
        personPresent = 1
      elif named_entities[i][1] == 'B-ORGANIZATION':
        organizationPresent = 1
      elif 'GPE' in named_entities[i][1]:
        gpePresent = 1
      elif 'LOCATION' in named_entities[i][1]:
        locationPresent = 1
    #print(named_entities)
    #If named entities are present then they have to be flagged in 4 categories
    #Person,Organization,GPE
    #LOCATION, NUMBER
    return np.array([personPresent,organizationPresent,gpePresent,locationPresent,text])

def replaceNamedEntities(text):
  named_entities = getNamedEntities(text)
  for i in range(0,len(named_entities)):
    #print(named_entities[i][0])
    if 'PERSON' in named_entities[i][1]:
      text = text.replace(named_entities[i][0],"")
    elif 'ORGANIZATION' in named_entities[i][1]:
      text = text.replace(named_entities[i][0],"")
    elif 'GPE' in  named_entities[i][1]:
      text = text.replace(named_entities[i][0],"")
    elif 'LOCATION' in  named_entities[i][1]:
      text = text.replace(named_entities[i][0],"")
  return text

In [ ]:
text = "Trump Declares Victory heads home To face more russia scandal"
verifyIfEntitiesArePresent(text)

array(['1', '0', '0', '0',
       'Trump Declares Victory heads home To face more russia scandal'],
      dtype='<U61')

In [ ]:
#import time
#code to generate and save named entities features into files on drive
import numpy as np
import time
start = time.time()
#with Pool(2) as pool:
t = df_subset['news_text'].apply(verifyIfEntitiesArePresent)


#print(np.array(t))
df_split = t.apply(pd.Series)

# Renaming columns
df_split.columns = ['PERSON','ORGANIZATION','GPE','LOCATION','NEWS_TEXT']

# Merging with the original DataFrame (Optional)
#df_final = pd.concat([df, df_split], axis=1).drop(columns=['Values'])

print(df_split)
end = time.time()
print(end-start)
df_split.to_csv('/content/drive/MyDrive/COMS574-Introduction_To_Machine_Learning/Named_Entities.csv', index=False)

KeyboardInterrupt: 

In [ ]:
#This is a crude way to remove named entities we dont want models to learn patterns based on particular named entities.
def removeWordsWhichHaveCaps(sentence):
    words = sentence.split()
    #print(words)
    #print(' '.join(['' if word[0].isupper() else word for word in words]))
    return ' '.join(['' if word[0].isupper() else word for word in words])

In [ ]:
sent = removeWordsWhichHaveCaps('L.L. bean becomes latest company to ban gun sales to people under 21 the company changed its policy in response to the Parkland shooting.')
print(sent)

 bean becomes latest company to ban gun sales to people under 21 the company changed its policy in response to the  shooting.


In [ ]:
import sys
#function to remove unicode characters
def remove_unicode(text):
    #print(text)
    return re.sub(r'[^\x00-\x7F]+', '', text)

#function to remove stopwords
def remove_stopwords(text):
    #text = "This is a simple sentence that contains some stop words."

    # Split the text into words
    words = text.split()

    # Get the list of stop words in English
    stop_words = set(stopwords.words('english'))

    # Remove stop words from the text
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Join the filtered words back into a sentence
    filtered_text = ' '.join(filtered_words)
    return filtered_text

#function to perform stemming and lemmatization.
def performLemmatizationAndStemming(text):
    words = text.split()

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = []
    for word in words:
        try:
            lemmatized_words.append(lemmatizer.lemmatize(word,pos='v'))
        except:
            print('exception thrown')
            print(word)
            sys.exit()
    lemmatized_words = [lemmatizer.lemmatize(word, pos='v') for word in words]  # Assuming verbs for simplicity

    # Stemming
    #stemmer = PorterStemmer()

    #stemmed_words = [stemmer.stem(word) for word in lemmatized_words]
    #stemmed_words = ' '.join(stemmed_words)
    lemmatized_words = ' '.join(lemmatized_words)
    return lemmatized_words


NameError: name 'stop_words' is not defined

In [ ]:
#crudely remove all named entities
#df_subset['news_text'] = df_subset['news_text'].apply(removeWordsWhichHaveCaps)
print(df_subset['news_text'])

0        John kerry laments ‘4 terribly lost Years’ of ...
1        hundreds of D.C.-Area students stage gun viole...
2        Watergate figure John dean says rick Gates' te...
3        Bernie sanders taps African-American organizer...
4        Obama hits Birthers in white house Corresponde...
                               ...                        
58692    12 sensational winter salads summer's not the ...
58693    drew brees is a __________! there weren't many...
58694    the 10 most ridiculous ice cube trays ever mad...
58695    savory bourbon recipes give us another way to ...
58696    Kristin Davis' spaghetti and Meatballs recipe ...
Name: news_text, Length: 58697, dtype: object


In [ ]:
def remove_nondictionary_words(sent):
    tokens = sent.split()
    return " ".join(['' if token not in setwords else token for token in tokens])

In [ ]:
#not starting with A-Z or 0-9 delete it
df_subset['news_text'] = df_subset['news_text'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
#remove unicode
df_subset['news_text'] = df_subset['news_text'].apply(remove_unicode)
#df_subset['news_text'] = df_subset['news_text'].apply(remove_nondictionary_words)
#replace \n with ''
df_subset['news_text'] = df_subset['news_text'].str.replace(r'[\n]','',regex=True)

In [ ]:
#remove stop words
df_subset['news_text'] = df_subset['news_text'].apply(remove_stopwords)

In [ ]:
df_subset['news_text'] = df_subset['news_text'].apply(performLemmatizationAndStemming)

In [ ]:
# Function to extract named entities
def figure_out_named_entities(text):
    # List to store named entities
    named_entities = []

    # Process each string in the dataset
    #for text in data:
    print(text)
    doc = nlp(text)  # Process the text
    print(doc)
    entities = [(ent.text, ent.label_) for ent in doc.ents]  # Extract named entities and their labels
    #named_entities.append(entities)
    print(named_entities)
    return [entity_type[1] for entity_type in entities]
    #return entities

# Extract named entities from the dataset
def extract_named_entities(data):
  entities = extract_named_entities(data)

  # Print the results
  #for i, entity_list in enumerate(entities):
      #print(f"Text {i+1}:")
      #for entity in entity_list:
          #print(f"  - {entity[0]} ({entity[1]})")
  [entity_type[1] for entity_type in entities]

In [ ]:
df_subset['news_text'].head()

,news_text
0,John kerry lament 4 terribly lose Years climat...
1,hundreds DCArea students stage gun violence pr...
2,Watergate figure John dean say rick Gates test...
3,Bernie sanders tap AfricanAmerican organizer c...
4,Obama hit Birthers white house Correspondents ...


In [ ]:
df_subset['news_text'] = df_subset['news_text'].str.lower()


In [ ]:
df_subset['news_text'].head()

,news_text
0,john kerry lament 4 terribly lose years climat...
1,hundreds dcarea students stage gun violence pr...
2,watergate figure john dean say rick gates test...
3,bernie sanders tap africanamerican organizer c...
4,obama hit birthers white house correspondents ...


In [ ]:
#get bag of words

unique_words = set(' '.join(df_subset['news_text']).split())
#print(df['news_text'][0])
print(len(unique_words))

57758


In [ ]:

categories = set(df_subset['category'].unique())


In [ ]:
df_subset = df_subset.sample(frac =1.0, random_state=42);

In [ ]:
[category for category in df_subset['category'].head()]

['POLITICS', 'ENTERTAINMENT', 'FOOD & DRINK', 'POLITICS', 'CRIME']

In [ ]:
categories_list = list(categories)
categories_index = [categories_list.index(category) for category in df_subset['category']]

,link,headline,category,short_description,authors,date,news_text
1856,https://www.huffingtonpost.com/entry/ben-sasse...,Republican Senator Asks If Trump Is Recanting ...,POLITICS,Ben Sasse questions whether Trump is failing t...,Ed Mazza,2017-10-12,NaN
45476,https://www.huffingtonpost.com/entry/moonlight...,'Moonlight' Saw Its Best Box-Office Totals Yet...,ENTERTAINMENT,"It's the largest Oscar bump since ""The Artist""...",Matthew Jacobs,2017-03-06,NaN
55060,https://www.huffingtonpost.com/entry/5-worth-o...,$5 Worth of Food Around the World,FOOD & DRINK,While it's no struggle to find expensive resta...,"The Daily Meal, Contributor\nAll things food a...",2014-04-07,NaN
612,https://www.huffingtonpost.com/entry/jared-kus...,Democrats Push White House To Explain Why Jare...,POLITICS,Democrats on the House Oversight Committee say...,Dominique Mosbergen,2017-06-22,NaN
28619,https://www.huffingtonpost.com/entry/antonio-s...,9-Year-Old Fatally Shot In Chicago Yard,CRIME,,Joseph Erbentraut,2014-08-21,NaN


In [ ]:
array_df_subset = df_subset['news_text'].astype(str).values
list_df_subset = list(array_df_subset)

In [ ]:
list_df_subset[0:5]

['white house treat halloween year ghost goblins scratch 1600 pennsylvania avenue trick treat rout',
 'tom jones honor elvis presley first video single new cd long lose suitcase beloved singer record artist sir tom joneshas new 13 song cd long lose suitcase scurve records albums first single video elvis presley blue man would become jones friend 1960s',
 'beyond chocolate create valentines day aphrodisiacs connections food sex ordinary come clich',
 'school shootings wont stop go wrong folks leave say fewer gun fewer gun crimes folks right say gun safety people would abuse gun',
 '9yearold fatally shoot chicago yard']

In [ ]:
array_df_subset

array([list(['White', 'House', 'Bars', 'Reporters', 'TrumpKim', 'Dinner', 'Vietnam', 'Reporters', 'several', 'outlets', 'include', 'Associated', 'Press', 'Reuters', 'Bloomberg', 'exclude', 'event']),
       list(['National', 'Rifle', 'Association', 'Endorses', 'Donald', 'Trump', 'President', 'Republican', 'candidate', 'back', 'ban', 'assault', 'weapons']),
       list(['Americans', 'Think', 'Legal', 'Immigrants', 'Good', 'Country', '35', 'want', 'level', 'legal', 'immigration', 'decrease']),
       ...,
       list(['10', 'Amazing', 'Vegan', 'GingerBased', 'Recipes', 'Western', 'cook', 'bake', 'ginger', 'perhaps', 'best', 'know', 'role', 'treat', 'like', 'gingersnaps', 'gingerbread', 'antiinflammatory', 'spice', 'versatile', 'healthy']),
       list(['Christopher', 'Walkens', 'Scallop', 'Recipe', 'Arrived', 'Please', 'recipe', 'justice', 'read', 'Christopher', 'Walkens', 'voice']),
       list(['Frozen', 'Whipped', 'Cream', 'Brilliant', 'Hot', 'Cocoa', 'Kitchen', 'Hack', 'idea', 'brill

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
vectorizer = CountVectorizer(binary=True)  # binary=True means 1 if word is present, 0 otherwise
X = vectorizer.fit_transform(list_df_subset)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, categories_index, test_size=0.2, random_state=42)

# Train Naive Bayes classifier
model = BernoulliNB()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.59


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.0005)

# Fit and transform the text data
tfidf_matrix = vectorizer.fit_transform(df_subset['news_text'])

# Convert to DataFrame for readability
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())


,abandon,abort,abroad,accept,access,accid,accomplish,accord,account,across,...,yesterday,yet,yoga,york,youd,young,younger,youth,zero,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.407325,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58692,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
58693,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
58694,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
58695,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.naive_bayes import MultinomialNB
vectorizer = CountVectorizer()  # binary=True means 1 if word is present, 0 otherwise
X = vectorizer.fit_transform(list_df_subset)
#X = tfidf_df;
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, categories_index, test_size=0.2, random_state=42)

# Train Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)

# Predictions
y_pred = model.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")


Accuracy: 0.66


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.0005)

# Fit and transform the text data
tfidf_matrix = vectorizer.fit_transform(df_subset['news_text'])

# Convert to DataFrame for readability
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())


In [ ]:
feature_names = vectorizer.get_feature_names_out()
len(feature_names)

4911

In [ ]:
from sklearn.decomposition import PCA
pca_model = PCA(n_components=0.75)
tfidf_df_reduced = pca_model.fit_transform(tfidf_df)


In [ ]:
len(tfidf_df_reduced[0])

4005

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tfidf_df_reduced, categories_index, test_size=0.2, random_state=42)

# Define the kernels to be used
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
kernel_chosen = kernels[0]
clf = SVC(kernel=kernel_chosen)

# Train the classifier
clf.fit(X_train, y_train)

# Make predictions on the test set
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)

# Print the results
print(f"Accuracy with {kernel_chosen} kernel: {accuracy:.2f}")

Accuracy with linear kernel: 0.65


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_df_reduced, categories_index, test_size=0.2, random_state=42)

# Create and train the logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.65


In [ ]:
with open('/content/drive/MyDrive/COMS574-Introduction_To_Machine_Learning/output_filtered.json', 'w') as file:
    # Iterate through each row and write it as a JSON object
    for index, row in df[['headline','short_description']].iterrows():
        # Convert the row to a dictionary
        row_dict = row.to_dict()
        # Write the dictionary to the file as a JSON object
        json.dump(row_dict, file)
        file.write('\n')  # To separate each JSON object by a newline

print("Data written to output.json")

Data written to output.json


In [ ]:
import spacy

# Load the pre-trained model
nlp = spacy.load("en_core_web_sm")

# Sample dataset (list of strings)
data = [
    "Apple is looking to buy a startup in the UK for $1 billion.",
    "Elon Musk is the CEO of Tesla and SpaceX.",
    "Microsoft's headquarters are located in Redmond, Washington.",
    "The Eiffel Tower is located in Paris, France."
]

# Function to extract named entities
def extract_named_entities(data):
    # List to store named entities
    named_entities = []

    # Process each string in the dataset
    for text in data:
        doc = nlp(text)  # Process the text
        entities = [(ent.text, ent.label_) for ent in doc.ents]  # Extract named entities and their labels
        named_entities.append(entities)

    return named_entities

# Extract named entities from the dataset
entities = extract_named_entities(data)

# Print the results
for i, entity_list in enumerate(entities):
    print(f"Text {i+1}:")
    for entity in entity_list:
        print(f"  - {entity[0]} ({entity[1]})")

Text 1:
  - Apple (ORG)
  - UK (GPE)
  - $1 billion (MONEY)
Text 2:
  - Elon Musk (PERSON)
  - Tesla (ORG)
Text 3:
  - Microsoft (ORG)
  - Redmond (GPE)
  - Washington (GPE)
Text 4:
  - The Eiffel Tower (LOC)
  - Paris (GPE)
  - France (GPE)


In [ ]:
array_2d = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])

# Convert 2D array to DataFrame
df = pd.DataFrame(array_2d)

# If you want to specify column names:
column_names = ['col1', 'col2', 'col3']
df = pd.DataFrame(array_2d, columns=column_names)

# If you want to specify index names:
index_names = ['row1', 'row2', 'row3']
df = pd.DataFrame(array_2d, index=index_names)

# If you want to specify both column and index names:
df = pd.DataFrame(array_2d, columns=column_names, index=index_names)

print(df)

      col1  col2  col3
row1     1     2     3
row2     4     5     6
row3     7     8     9


In [ ]:
import nltk
from nltk.corpus import words

nltk.download('words')
word_list = words.words()

def check_word_nltk(word):
    return word.lower() in word_list

print(check_word_nltk("example"))
print(check_word_nltk("nonexistentword"))

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True
False
